### Imports

In [0]:
from datetime import datetime, timezone
import pandas as pd
from delta.tables import DeltaTable
from pyspark.sql.types import StringType
from pyspark.sql.functions import (col,
                                   lit,
                                   max as _max,
                                   concat_ws,
                                   sha2)

### Sample Data

In [0]:
data = {
    "name_id": [1, 2],
    "name": ["Pedro", "John"],
    "dob": ["1989-01-25", "1990-01-25"],
    "town": ["Adelaide", "Melbourne"],
    "updated_at": ["2024-08-30 03:56:12", "2024-08-30 03:56:12"],
}

source = pd.DataFrame(data)
df = spark.createDataFrame(source)
df.write.format("delta").mode("overwrite").saveAsTable("names")

### Sink SCD Table Definition

In [0]:
%sql
-- TRUNCATE TABLE names_scd2

In [0]:
# create sink table empty
(
    DeltaTable.createIfNotExists(spark)
    .tableName("names_scd2")
    .addColumn("hash_key", "STRING")
    .addColumn("name_id", "INT")
    .addColumn("name", "STRING")
    .addColumn("dob", "DATE")
    .addColumn("town", "STRING")
    .addColumn("from_date", "TIMESTAMP")
    .addColumn("to_date", "TIMESTAMP")
    .addColumn("current", "BOOLEAN")
    .addColumn("updated_at", "TIMESTAMP")
    .execute()
)

### hash_Key Logic

In [0]:
def createHashKey(df):
    columns = [column for column in df.columns]
    s_columns = [f"s_{col}" for col in columns]

    # Create string copies for all columns
    for col in df.columns:
        df = df.withColumn(f"s_{col}", df[col].cast(StringType()))
    # concatenate values
    df = df.withColumn(
        "concatenated_values", concat_ws("", *s_columns)
    )

    # Compute Sha256
    df = df.withColumn("hash_key", sha2(df["concatenated_values"], 256))

    columns.append("hash_key")
    df = df.select(columns)
    return df

### SCD2 Merge

In [0]:
def merge_scd2(updated_df, sink_table):
    changed_records = (
        updated_df.alias("updates")
        .join(sink_table.toDF().alias("scd2"), "name_id")
        .where(
            """
                           `scd2`.`current` = true
                           and
                           `updates`.`hash_key` <> `scd2`.`hash_key`
                           """
        )
    )
    print("changed records")
    display(changed_records)

    # Stage changes
    # new records will be inserted new records and new versions
    # to_date will be null and from_date will be last_updated, for completely
    # changed records that will be updated, current flag to false and to_date updated to last_updated
    staged_updates = changed_records.selectExpr(
        "null as merge_key", "updates.*"
    ).union(  # changed updated records
        updates_df.selectExpr("name_id as merge_key", "*")
    )  # new records either new version
    print("staged_updates")
    display(staged_updates)

    # merge intp scd2
    now = datetime.now(timezone.utc).isoformat()
    staged_updates = staged_updates.withColumn("updated_at", lit(now))

    (
        sink_table.alias("scd2")
        .merge(staged_updates.alias("staged_updates"), "scd2.name_id = merge_key")
        .whenMatchedUpdate(
            condition="""
                    `scd2`.`current` = true
                    and `staged_updates`.`hash_key` <> `scd2`.`hash_key`
                    """,
            set={"current": "false", "to_date": "staged_updates.updated_at"},
        )
        .whenNotMatchedInsert(
            values={
                "hash_key": "staged_updates.hash_key",
                "name_id": "staged_updates.name_id",
                "name": "staged_updates.name",
                "dob": "staged_updates.dob",
                "town": "staged_updates.town",
                "current": "true",
                "from_date": "staged_updates.updated_at",
                "to_date": "null",
            }
        )
        .execute()
    )

### Watermark tracking

In [0]:
watermark = "2000-01-01"
source_table = spark.read.table("names")
sink_table = DeltaTable.forName(spark,'names_scd2')

updates_df = source_table.filter(col("updated_at") > watermark)
watermark = (source_table
             .select(['updated_at'])
             .agg(_max(col('updated_at')))
             .collect()[0][0]
             )
updates_df = createHashKey(updates_df)             
display(updates_df)

name_id,name,dob,town,updated_at,hash_key
1,Pedro,1989-01-25,Adelaide,2024-08-30 03:56:12,e0fb8e44725422f9c3a1604d2233031e203a4e2638537df643af970d5cc91c4e
2,John,1990-01-25,Melbourne,2024-08-30 03:56:12,9219f60be71a80a1d40947c119b841c4dc869eadb748ae6ca56fb897f9cbff36


### First Run with inserts only

In [0]:
merge_scd2(updates_df,sink_table)

changed records


name_id,name,dob,town,updated_at,hash_key,hash_key,name,dob,town,from_date,to_date,current,updated_at


staged_updates


merge_key,name_id,name,dob,town,updated_at,hash_key
1,1,Pedro,1989-01-25,Adelaide,2024-08-30 03:56:12,e0fb8e44725422f9c3a1604d2233031e203a4e2638537df643af970d5cc91c4e
2,2,John,1990-01-25,Melbourne,2024-08-30 03:56:12,9219f60be71a80a1d40947c119b841c4dc869eadb748ae6ca56fb897f9cbff36


### Updates to raw table

In [0]:
source_table = DeltaTable.forName(spark,'names')
display(source_table.toDF())

(
    source_table
    .update(
        condition=col('name_id') == 1,
        set = {'town': lit('Sao Paulo'),
               'updated_at': lit(datetime.now(timezone.utc).isoformat())}
    )
)

source_table = DeltaTable.forName(spark,'names')
display(source_table.toDF())


name_id,name,dob,town,updated_at
1,Pedro,1989-01-25,Adelaide,2024-08-30 03:56:12
2,John,1990-01-25,Melbourne,2024-08-30 03:56:12


name_id,name,dob,town,updated_at
1,Pedro,1989-01-25,Sao Paulo,2024-08-30T08:27:30.757054+00:00
2,John,1990-01-25,Melbourne,2024-08-30 03:56:12


In [0]:
%sql
insert into names (name_id,name,dob,town,updated_at)
values (3,"richard",'2001-01-04','Montreal',CURRENT_TIMESTAMP()),
       (4,"Maria",'2001-01-04','New York',CURRENT_TIMESTAMP())

num_affected_rows,num_inserted_rows
2,2


2024-08-30 03:56:12


In [0]:
source_table = spark.read.table("names")
sink_table = DeltaTable.forName(spark,'names_scd2')

updates_df = source_table.filter(col("updated_at") > watermark)
display(updates_df)

print(f'original watermark {watermark}')
watermark = (source_table
             .select(['updated_at'])
             .agg(_max(col('updated_at')))
             .collect()[0][0]
             )
print(f'updated watermark {watermark}')
updates_df = createHashKey(updates_df)

name_id,name,dob,town,updated_at
3,richard,2001-01-04,Montreal,2024-08-30 08:27:38.234
4,Maria,2001-01-04,New York,2024-08-30 08:27:38.234
1,Pedro,1989-01-25,Sao Paulo,2024-08-30T08:27:30.757054+00:00


original watermark 2024-08-30 03:56:12
updated watermark 2024-08-30T08:27:30.757054+00:00


In [0]:
merge_scd2(updates_df,sink_table)

changed records


name_id,name,dob,town,updated_at,hash_key,hash_key,name,dob,town,from_date,to_date,current,updated_at
1,Pedro,1989-01-25,Sao Paulo,2024-08-30T08:27:30.757054+00:00,eb808950db8a71147ade44eba4a34bffe82f6fc682105cdcc705a42da03077b1,e0fb8e44725422f9c3a1604d2233031e203a4e2638537df643af970d5cc91c4e,Pedro,1989-01-25,Adelaide,2024-08-30T08:27:24.111426Z,null,true,null


staged_updates


merge_key,name_id,name,dob,town,updated_at,hash_key
null,1,Pedro,1989-01-25,Sao Paulo,2024-08-30T08:27:30.757054+00:00,eb808950db8a71147ade44eba4a34bffe82f6fc682105cdcc705a42da03077b1
3,3,richard,2001-01-04,Montreal,2024-08-30 08:27:38.234,5d26aa0abbb6047ee425bf745b154389cb92718663594daab5c03daa0de38e11
4,4,Maria,2001-01-04,New York,2024-08-30 08:27:38.234,f45c6b5479755b3b9078e51861edf5309994e90036d1bd1462474695867e4ef1
1,1,Pedro,1989-01-25,Sao Paulo,2024-08-30T08:27:30.757054+00:00,eb808950db8a71147ade44eba4a34bffe82f6fc682105cdcc705a42da03077b1


### Final Result

In [0]:
display(sink_table.toDF())

hash_key,name_id,name,dob,town,from_date,to_date,current,updated_at
e0fb8e44725422f9c3a1604d2233031e203a4e2638537df643af970d5cc91c4e,1,Pedro,1989-01-25,Adelaide,2024-08-30T08:27:24.111426Z,2024-08-30T08:29:28.974068Z,false,null
eb808950db8a71147ade44eba4a34bffe82f6fc682105cdcc705a42da03077b1,1,Pedro,1989-01-25,Sao Paulo,2024-08-30T08:29:28.974068Z,null,true,null
9219f60be71a80a1d40947c119b841c4dc869eadb748ae6ca56fb897f9cbff36,2,John,1990-01-25,Melbourne,2024-08-30T08:27:24.111426Z,null,true,null
5d26aa0abbb6047ee425bf745b154389cb92718663594daab5c03daa0de38e11,3,richard,2001-01-04,Montreal,2024-08-30T08:29:28.974068Z,null,true,null
f45c6b5479755b3b9078e51861edf5309994e90036d1bd1462474695867e4ef1,4,Maria,2001-01-04,New York,2024-08-30T08:29:28.974068Z,null,true,null
